### Requirements

**Gantt chart with projects and project-specific tasks; created through a JSON file.**

Task attributes:
- start and end dates
- parent-project 
- description
- milestones
- status

Visuals should include:
- color-coding according to yearly quartile
- month and quarter axis frequency 
- task description at the bottom 
- milestones



----
#### Input

In [33]:
projects = {
    '1' : {
        '1': {
            'start': '2024-05-01',
            'end': '2024-06-15',
            'description': 'Systematic review on methodologies for seizure forecasting: contact Epilepsia editor, redo forest plot, rewrite document, appendix tables, and submit with B-on agreement.',
            'status': 'I',
        },
        '2': {
            'start': '2024-05-01',
            'end': '2024-06-30',
            'description': 'Bioentrepreneurship project.',
            'status': 'I',
        },
        '3': {
            'start': '2024-05-01',
            'end': '2024-06-15',
            'description': 'Systematic review on methodologies for seizure forecasting: contact Epilepsia editor, redo forest plot, rewrite document, appendix tables, and submit with B-on agreement.',
            'status': 'I',
        },
    }   
}

----
#### We need to transform the json into something like:

| task | projects | start | end | status | milestone | description |
| ---- | -------- | ----- | --- | ------ | --------- | ----------- | 
| task 1.1 | project 1 | 2024-05-01 | 2024-06-15 | I | ? | ? |

In [34]:
import pandas as pd

gantt = pd.DataFrame(columns=['task', 'project', 'start', 'end', 'description', 'status'])

for p, project in enumerate(projects):
    for t, task in enumerate(projects[project]):

        att = []
        for key in projects[project][task]:
            att += [projects[project][task][key]]

        gantt.loc[len(gantt)] = [f'Task {p+1}.{t+1}', f'Project {p+1}'] + att


# convert str to datetime
gantt['start'] = pd.to_datetime(gantt['start'])
gantt['end'] = pd.to_datetime(gantt['end'])

# get start and end of timeline
gantt['first'] = gantt['start'].min()
gantt['last'] = gantt['end'].max()

# get yearly quarter
gantt['quarter'] = gantt['start'].apply(lambda x:(x.month-1)//3 + 1).values
gantt


,task,project,start,end,description,status,first,last,quarter
0,Task 1.1,Project 1,2024-05-01,2024-06-15,Systematic review on methodologies for seizure...,I,2024-05-01,2024-06-30,2
1,Task 1.2,Project 1,2024-05-01,2024-06-30,Bioentrepreneurship project.,I,2024-05-01,2024-06-30,2
2,Task 1.3,Project 1,2024-05-01,2024-06-15,Systematic review on methodologies for seizure...,I,2024-05-01,2024-06-30,2


In [40]:
import plotly.express as px
from constants import COLOR_PALETTE

# add backgorund color to each task's timeline 
fig = px.timeline(gantt, x_start='first', x_end='last', y='task', color_discrete_sequence=['#F4F4F4'])


fig.add_trace(px.timeline(gantt, x_start='start', x_end='end', y='task', color='quarter').data[0])


fig.update_yaxes(autorange="reversed", visible=False, showticklabels=False)
fig.update_xaxes(visible=False, showticklabels=False)

fig.update_layout(
    plot_bgcolor='white',
    coloraxis_showscale=False
)
fig.show()